# Schema's in MongoDB

MongoDB database: collections met documenten

Collection: documenten "van dezelfde soort"

Document bepaalt **eigen schema**: namen en types van velden

*Hoe kunnen we afdwingen dat documenten in een collection **hetzelfde schema** (namen, types) hebben?*

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"
pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

Initialisatie van de database. Het resultaat moet 0 zijn.

In [ ]:
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]

contacts = db.contacts
contacts.drop()
os.system('mongoimport -d ' + dbname + ' -c contacts adressen.json')

Toevoegen van een "vreemd" document

In [ ]:
contacts.insert_one({"kleur": "groen", "prijs": 400, "beschrijving": "fiets"})

Zoeken op *type* van een veld:

In [ ]:
list(contacts.find({"kleur": {"$type": "string"}}))

## MongoDB collection-validator (schema)

* MongoDB query met namen en types

* met de mogelijkheid van alternatieven

* *minimale eis* aan documenten in collection (overige velden vrij)

In [ ]:
contact_schema = {"name": {"$type": "string"},
                  "$or": [{"email": {"$type": "string"}},
                          {"tel": {"$type": "string"}}

                         ]}

In [ ]:
list(contacts.find(contact_schema))

Vinden van documenten die *niet* aan schema voldoen:

In [ ]:
list(contacts.find({"$nor": [contact_schema]}))

Koppelen van schema (validator) aan collection:

In [ ]:
db.command("collMod", "contacts", validator=contact_schema)

toevoegen van niet-valide document:

In [ ]:
contacts.insert_one({"naam": "Joke Baarsma", "email": "jokeb@friends.org"})

Dat is handiger op te lossen met exception handling

In [ ]:
try:
    contacts.insert_one({"naam": "Joke Baarsma", "email": "jokeb@friends.org"})
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

toevoegen van valide document:

In [ ]:
try:
    contacts.insert_one({"name": "Joke Baarsma", "email": "jokeb@friends.org"})
except pymongo.errors.WriteError as s:
    print("Document not inserted: " + str(s))
else:
    print("insert OK")

Oefenen met MongoDB schema's / validators